# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#abrir navegador
navegador = webdriver.Chrome()

#Dolar

# entrar no google
navegador.get("https://google.com")
#navegador.get("https://dolarhoje.com")

#localizar a caixa de busca no site e escrever nela
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao dolar")

#apertar ENTER na caixa de busca
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#pegar cotação
#dolar = navegador.find_element('xpath','//*[@id="nacional"]').get_attribute('value')
dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Euro
navegador.get("https://google.com")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao euro")
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")
ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
ouro = ouro.replace(",",".")

navegador.quit()

print(dolar)
print(euro)
print(ouro)


5.1772
5.310532903
297.88


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [19]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [20]:
#atualizando a cotação
tabela.loc[tabela["Moeda"] == "Dólar","Cotação"] = float(dolar)
tabela.loc[tabela["Moeda"] == "Euro","Cotação"] = float(euro)
tabela.loc[tabela["Moeda"] == "Ouro","Cotação"] = float(ouro)

#atualizando preço de compra
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#atualizando preço de venda
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.177200,5177.148228,1.40,7248.007519
1,Carro Renault,4500.00,Euro,5.310533,23897.398064,2.00,47794.796127
2,Notebook Dell,899.99,Dólar,5.177200,4659.428228,1.70,7921.027988
3,IPhone,799.00,Dólar,5.177200,4136.582800,1.70,7032.190760
4,Carro Fiat,3000.00,Euro,5.310533,15931.598709,1.90,30270.037547
5,Celular Xiaomi,480.48,Dólar,5.177200,2487.541056,2.00,4975.082112
6,Joia 20g,20.00,Ouro,297.880000,5957.600000,1.15,6851.240000


### Agora vamos exportar a nova base de preços atualizada

In [21]:
tabela.to_excel("Produtos_Atualizados.xlsx", index = False)